# Lab 2 - P4 Program Building Blocks with the PNA Architecture
<div style="text-align: justify;">
This lab will walk you through creating an experiment that contains a P4-DPDK programmable pipeline. Specifically, the experiment deploys a slice in a single site with one node: server1. The topology is shown in the figure below. This lab provides guidance to the process of building the P4 coding blocks compatible with the PNA architecture that will be compiled into a DPDK pipeline. In this lab, we will be implementing a simple application in which a packet is sent from a server and forwarded to the client. 
<figure style="text-align: center;">
    <img src="./labs_files/lab2/figs/0_00_fabric_topology.png" width="550px"><br>

# Introduction to P4 architecture

<div style="text-align: justify;">
A P4 architecture represents a programming model that outlines the capabilities of a target to process a P4 pipeline. Manufacturers provide for each specific hardware or P4 target, the architecture and a P4 compiler. P4 programs are specifically written for a particular P4 architecture and can be applied to any targets compatible with the same architecture. The compilation of these P4 programs yields two key elements: a data plane configuration implementing the forwarding logic based on the input program and an API facilitating the control plane's management of the data plane object states [<a href="#References">1</a>]. </div>  <br>

## The PNA architecture

<div style="text-align: justify;">
The Portable NIC Architecture (PNA) is a P4 architecture that defines the structure and common capabilities for programmable NICs. PNA's primary objective is to provide P4 capabilities for deploying packet processing functions on NICs. It enables P4 programmers to create portable P4 programs that can be compiled and executable across various NIC devices. PNA has four P4 programmable blocks (main parser, pre-control, main control, and main deparser), and several fixed-function blocks, as shown in Figure 1. The host-to-net and net-to-host externs allow executing functions on the domain-specific accelerators such as encrypting or decrypting IPsec payload. The message processing is responsible for converting between large messages in host memory and network size packets on the network and for dealing with one or more host operating systems, drivers, and/or message descriptor formats in host memory[<a href="#References">2,3</a>]. </div>   <br> 

<figure style="text-align: center;">
  <img src="./labs_files/lab2/figs/0_intro_1_pna.png" width="550" style="display: block; margin: 0 auto;">
  <figcaption>Figure 1. Portable NIC architecture (PNA) [<a href="#References">2</a>].</figcaption>
</figure>

## Programmable parser

<div style="text-align: justify;">
Packets arriving from a network port first go through a Main Parser and a Pre-Control which will be discussed in the next subsection. The programmable parser permits the programmer to define the headers (according to custom or standard protocols) and to describe how the headers should be processed. The parser de-encapsulates the headers, converting the original packet into a parsed representation of the packet. The programmer declares the headers that must be recognized and their order in the packet. The parser can be represented as a state machine without cycles (direct acyclic graph), with one initial state (start) and two final states (accept or reject) [<a href="#References">2,3</a>]. </div>   <br> 

## Programmable deparser

<div style="text-align: justify;">
The deparser assembles back the packet and serializes it for transmission. The programmer specifies the headers to be emitted by the deparser. When assembling the packet, the deparser emits the specified headers followed by the original payload of the packet [<a href="#References">2,3</a>]. </div>   <br> 

## P4 program mapping to the PNA

<div style="text-align: justify;">
The P4 program used in this lab is separated into different files. Figure 2 shows the PNA architecture and its associated P4 files. These files are as follows: </div>   <br> 

•	headers.p4: this file contains the packet headers’ and the metadata’s definitions.<br>
•	parser.p4: this file contains the implementation of the programmable parser.<br>
•	precontrol.p4: this file contains the pre-control block that optionally includes table look-ups.<br>
•	control.p4: this file contains the main control block.<br>
•	deparser.p4: this file contains the deparser logic that describes how headers are emitted.<br>
•	main.p4: this file contains the starting point of the program and invokes the other files. This file must be compiled.<br>

<figure style="text-align: center;">
  <img src="./labs_files/lab2/figs/0_intro_2_p4.png" width="550" style="display: block; margin: 0 auto;">
  <figcaption>Figure 2. Mapping of P4 files to the PNA’s components.</figcaption>
</figure>

# Step 1:  Configuring the environment

Before running this notebook, you will need to configure your environment using the [Configure Environment](../../../configure_and_validate.ipynb) notebook. Please stop here, open and run that notebook, then return to this notebook.

If you are using the FABRIC JupyterHub many of the environment variables will be automatically configured for you.  You will still need to set your bastion username, upload your bastion private key, and set the path to where you put your bastion private key. Your bastion username and private key should already be in your possession.  

If you are using the FABRIC API outside of the JupyterHub you will need to configure all of the environment variables. Defaults below will be correct in many situations but you will need to confirm your configuration.  If you have questions about this configuration, please contact the FABRIC admins using the [FABRIC User Forum](https://learn.fabric-testbed.net/forums/) 

More information about accessing your experiments through the FABRIC bastion hosts can be found [here](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/).

# Step 2: Importing the FABlib library

In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()

# Step 3: Creating the experiment slice

The following creates a node with basic compute and networking capabilities. You build a slice by creating a new slice and adding resources to the slice. After you build the slice, you must submit a request for the slice to be instantiated.   

### Step 3.1: Creating a slice
The code below creates a new slice with the name "P4DPDK_lab2_vnic"

In [2]:
slice = fablib.new_slice(name="P4DPDK_lab2_vnic")

### Step 3.2: Defining the sites
The code below requests a random site from FABRIC based on the condition that the following resources are available:

<ul>
    <li> 4 CPU cores</li>
    <li> 8GB RAM </li>
    <li> 20GB disc size
</ul>

In [3]:
site1= fablib.get_random_sites(count=1, filter_function=lambda x: x['cores_available'] > 4 and x['ram_available'] > 8 and x['disk_available'] > 20)[0]

print (f'The selected site is {site1}')

The selected site is NEWY


### Step 3.3: Creating the nodes
The code below creates one node (server1) which uses the following:
<ul>
    <li> 4 CPU cores</li>
    <li> 8GB RAM </li>
    <li> 20GB disc size </li>
    <li> Image: Ubuntu 20.04
</ul>

server1 will be created in site1

In [4]:
server1 = slice.add_node(name="server1", 
                      site=site1, 
                      cores=4, 
                      ram=8, 
                      disk=20, 
                      image='default_ubuntu_20')

### Step 3.4: Submitting the slice
The code below submits the slice. 
By default, the submit function will block until the node is ready and will display the progress of your slice being built.

In [5]:
slice.submit();


Retry: 8, Time: 185 sec


ID,5ec2837c-8df8-4856-abe3-987b7c72aebb
Name,P4DPDK_lab2_vnic
Lease Expiration (UTC),2024-09-03 18:29:40 +0000
Lease Start (UTC),2024-09-02 18:29:40 +0000
Project ID,8eaa3ec2-65e7-49a3-8c09-e1761141a6ad
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
fbe578b7-90d2-4f95-9c7a-65c7d1d3dc75,server1,4,8,100,default_ubuntu_20,qcow2,newy-w2.fabric-testbed.net,NEWY,ubuntu,2001:400:a100:3040:f816:3eff:fe9e:4178,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3040:f816:3eff:fe9e:4178,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


# Step 4: Installing the required packages
In this step, we will install the required packages to run the lab. Specifically, we will install the DPDK library, the P4 compiler (p4c), and all needed dependencies.

### Step 4.1 Getting the server node
The command below gets the fablib node that server1 is associated with.

In [6]:
server1 = slice.get_node(name="server1")

### Step 4.2 NAT64 setup
The code below checks if an IPv6 address is available to set up NAT64. We will upload the script [scripts/nat64.sh](./scripts/nat64.sh) to the all servers and execute it

In [7]:
from ipaddress import ip_address, IPv6Address

if type(ip_address(server1.get_management_ip())) is IPv6Address:
    server1.upload_file('scripts/nat64.sh', 'nat64.sh')
    stdout, stderr = server1.execute(f'chmod +x nat64.sh && ./nat64.sh', quiet=True)

### Step 4.3 Installing dependencies
The code below installs packages that are prerequisites to the upcoming installations and needed to run the lab experiments

In [8]:
stdout, stderr = server1.execute(f'sudo apt-get update', quiet = True)
stdout, stderr = server1.execute(f'sudo apt-get install -y build-essential python3-pip python3-pyelftools libnuma-dev pkg-config net-tools', quiet = True)
stdout, stderr = server1.execute(f'sudo pip3 install meson ninja', quiet = True)

### Step 4.4 Installing DPDK
The code below downloads, builds, and installs DPDK on all servers. In this lab, we are building a modified version of DPDK in which we enabled logs in the terminal to have a better understanding of the behavior of the built pipeline. Note that printing out logs on the terminal degrades performance. Therefore, with applications where high performance is needed, logging should be disabled.  

In [9]:
stdout, stderr = server1.execute(f'git clone http://dpdk.org/git/dpdk', quiet = True)
stdout, stderr = server1.execute(f'cd dpdk/lib/pipeline/ &&  sudo rm rte_swx_pipeline.c && sudo rm rte_swx_pipeline_internal.h', quiet = True)
server1.upload_file('scripts/rte_swx_pipeline.c','/home/ubuntu/dpdk/lib/pipeline/rte_swx_pipeline.c')
server1.upload_file('scripts/rte_swx_pipeline_internal.h','/home/ubuntu/dpdk/lib/pipeline/rte_swx_pipeline_internal.h')
stdout, stderr = server1.execute(f'cd dpdk &&  sudo meson build && cd build && sudo ninja && sudo ninja install && sudo ldconfig', quiet = True)
stdout, stderr = server1.execute(f'cd dpdk/examples/pipeline && sudo make', quiet=True)

### Step 4.5 Install p4c
The code below downloads and installs the p4c compiler needed to compile the p4 code into a DPDK pipeline

In [10]:
stdout, stderr = server1.execute('git clone https://github.com/CILab-USC/p4c.git', quiet = True)
stdout, stderr = server1.execute('sudo apt-get install -y cmake g++ git automake libtool libgc-dev bison flex libfl-dev libboost-dev libboost-iostreams-dev libboost-graph-dev llvm pkg-config python3 python3-pip tcpdump', quiet = True)
stdout, stderr = server1.execute('cd p4c && pip3 install --user -r requirements.txt && mkdir build && cd build && cmake .. && make -j4  && sudo make install', quiet = True)

# Step 5: Writing and compiling the P4 code
This section shows the steps required to write the P4 code. It discusses the individual coding blocks that form the PNA architecture.

## Step 5.1: Describing the components of the P4 program

### Step 5.1.1: Inspecting P4 code
Click on [main.p4](./labs_files/lab2/main.p4) to open the file in the editor.

<img src="./labs_files/lab2/figs/5_01_01.png" width="700px"><br>

The main.p4 file includes the starting point of the P4 program and other files that are specific to the language ```core.p4``` and to the architecture ```pna.p4```. To make the P4 program easier to read and understand, we separated the whole program into different files that correspond to the components of the PNA model. To use those files, the main file ```main.p4``` must include them first. For example, to use the parser, we need to include the parser.p4 file ```#include “parser.p4”```.

We will navigate through the files in sequence as they appear in the architecture.


### Step 5.1.2: Inspecting headers.p4
Click on [headers.p4](./labs_files/lab2/headers.p4) to open the file in the editor.

<img src="./labs_files/lab2/figs/5_01_02.png" width="700"><br>

The headers.p4 above shows the headers that will be used in our pipeline. We can see that the ethernet header is defined. We can also see how it is inserted in a structure ```struct headers```. The ```headers``` name will be used throughout the program when referring to the headers. Furthermore, the file shows how we can use ```typedef``` to provide an alternative name to a type.


### Step 5.1.3: Inspecting parser.p4
Click on [parser.p4](./labs_files/lab2/parser.p4) to open the file in the editor.

<img src="./labs_files/lab2/figs/5_01_03.png" width="700px"><br>

The figure above shows the content of the parser.p4 file. We can see that the parser is already written with the name ```MyParser```. This name will be used when defining the pipeline sequence.


### Step 5.1.4: Inspecting precontrol.p4
Click on [precontrol.p4](./labs_files/lab2/precontrol.p4) to open the file in the editor.

<img src="./labs_files/lab2/figs/5_01_04.png" width="700px"><br>

The figure above shows the content of the precontrol.p4 file. We can see that the parser is already written with the name ```PreControl```. This name will be used when defining the pipeline sequence. 


### Step 5.1.5: Inspecting control.p4
Click on [control.p4](./labs_files/lab2/control.p4) to open the file in the editor.

<img src="./labs_files/lab2/figs/5_01_05.png" width="700px"><br>

The figure above shows the content of the control.p4 file. We can see that the code is already written with the name ```MainControl```. This name will be used when defining the pipeline sequence. 

### Step 5.1.6: Inspecting deparser.p4
Click on [deparser.p4](./labs_files/lab2/deparser.p4) to open the file in the editor.

<img src="./labs_files/lab2/figs/5_01_06.png" width="700px"><br>

The figure above shows the content of the deparser.p4 file. We can see that the code is already written with the name ```MyDeparser```. We can see that the deparser is already written with one instruction that reassembles the packet.

## Step 5.2: Programming the pipeline sequence
Now it is time to write the pipeline sequence in the main.p4 program. 

Click on [main.p4](./labs_files/lab2/main.p4) to open the CLI file in the editor.

Write the following block of code at the end of the file:

    PNA_NIC(
    MyParser(),
    PreControl(),
    MainControl(),
    MyDeparser()
    ) main;

<img src="./labs_files/lab2/figs/5_02.png" width="700px"><br>

We can see here that we are defining the pipeline sequence according to the PNA architecture. First, we start with the parser and the pre-control. Afterwards, the main control block is included. Finally, we specify the deparser.

<hr>

Save the changes by pressing ```Ctrl+s```.

## Step 5.3: Compiling the P4 code
To upload all needed P4 codes and compile main.p4, issue the following command.

In [30]:
server1.upload_file('labs_files/lab2/headers.p4','headers.p4')
server1.upload_file('labs_files/lab2/parser.p4','parser.p4')
server1.upload_file('labs_files/lab2/precontrol.p4','precontrol.p4')
server1.upload_file('labs_files/lab2/control.p4','control.p4')
server1.upload_file('labs_files/lab2/deparser.p4','deparser.p4')
server1.upload_file('labs_files/lab2/main.p4','main.p4')
stdout, stderr = server1.execute(f'sudo p4c-dpdk --arch=pna main.p4 -o lab2.spec')
stdout, stderr = server1.execute(f'ls')

 sudo: unable to resolve host server1: Name or service not known
 control.p4
deparser.p4
dpdk
ethdev.io
headers.p4
host_tune.sh
lab2.cli
lab2.spec
main.p4
nat64.sh
p4c
parser.p4
precontrol.p4
run_pipeline.sh
set_topology.sh
setup.sh


The command above invokes the ```p4c-dpdk``` compiler to compile the ```main.p4``` program and generates the ```lab2.spec``` file which is a specification file needed to run the pipeline.

# Step 6: Running the P4-DPDK pipeline and the lab topology
This section shows the steps required to run the P4-DPDK along with building the lab topology. In this lab, the procedure is automated.

## Step 6.1: Uploading required scripts
The code below uploads to server1 the CLI and I/O scripts, along with other scripts needed to automate the process of running the pipeline and building the topology.

In [31]:
server1.upload_file('labs_files/lab2/lab2.cli','lab2.cli')
server1.upload_file('labs_files/lab2/ethdev.io','ethdev.io')
server1.upload_file('labs_files/lab2/run_pipeline.sh','run_pipeline.sh')
server1.upload_file('labs_files/lab2/set_topology.sh','set_topology.sh')
stdout, stderr = server1.execute(f'chmod +x run_pipeline.sh')
stdout, stderr = server1.execute(f'chmod +x set_topology.sh')

## Step 6.2: Opening a terminal
Launch a new terminal by opening a new tab and then select "terminal".

<img src="./labs_files/lab2/figs/6_02_terminal.gif" width="1000px"><br>

Copy the output of the command below and paste it into the terminal to enter server1.

In [16]:
server1.get_ssh_command()

'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3040:f816:3eff:fe9e:4178'

## Step 6.3: Running the P4-DPDK pipeline
Build and run the pipeline by typing the following command:

    sudo ./run_pipeline.sh
    
<img src="./labs_files/lab2/figs/6_03.png" width="650px"><br>

The ```run_pipeline.sh``` script is a shell script that automates the process of running the P4-DPDK pipeline. 

## Step 6.4: Building the lab topology
The code below uploads to server1 the CLI and I/O scripts, along with other scripts needed to automate the process of running the pipeline and building the topology.

In [27]:
stdout, stderr = server1.execute(f'sudo ./set_topology.sh')

 sudo: unable to resolve host server1: Name or service not known
Cannot create namespace file "/run/netns/h1": File exists
Cannot create namespace file "/run/netns/h2": File exists
 

The set_topology.sh script is a shell script that automates the process of building the lab topology. Two namespaces are built and configured in this step with a virtual device linked to each as shown in the figure below.

<img src="./labs_files/lab1/figs/6_04_ip.png" width="550px"><br>

# Step 7: Testing connectivity

### Step 7.1: Sending packets from h1 to h2
Test the connectivity between namespaces h1 and h2 using the ping command.

In [23]:
stdout, stderr = server1.execute(f'sudo ip netns exec h1 ping 192.168.10.2 -c 4')

PING 192.168.10.2 (192.168.10.2) 56(84) bytes of data.
64 bytes from 192.168.10.2: icmp_seq=1 ttl=64 time=1.30 ms
 sudo: unable to resolve host server1: Name or service not known
 64 bytes from 192.168.10.2: icmp_seq=2 ttl=64 time=0.146 ms
64 bytes from 192.168.10.2: icmp_seq=3 ttl=64 time=0.152 ms
64 bytes from 192.168.10.2: icmp_seq=4 ttl=64 time=0.147 ms

--- 192.168.10.2 ping statistics ---
4 packets transmitted, 4 received, 0% packet loss, time 3056ms
rtt min/avg/max/mdev = 0.146/0.435/1.298/0.497 ms


### Step 7.2: Stopping the DPDK pipeline
Stop the DPDK pipeline by pressing ```ctrl+c``` in the terminal running the pipeline.

<img src="./labs_files/lab2/figs/7_02.png" width="700px"><br>

Note that the logs in the terminal correspond to the code executed for packet processing. 

# Step 8: Delete the slice

This concludes Lab 2. Please delete your slice when you are done with your experiment.

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()
slice = fablib.get_slice(name="P4DPDK_lab2_vnic")
slice.delete()

# References

1.	S. Ibanez, “The p4-> netfpga workflow for line-rate packet processing”, Proceedings of the 2019 ACM/SIGDA International Symposium on Field-Programmable Gate Arrays, 2019.
2.	The P4 Language Consortium, “P4 Portable NIC Architecture (PNA)”, Version 0.5, 2021. [Online]. Available: https://p4.org/p4-spec/docs/PNA.html
3.	P4lang, “pna”, [Online]. Available: https://github.com/p4lang/pna/tree/main?tab=readme-ov-file